In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
import numpy as np
import csv
import scipy

In [9]:
id_eleccount = # Removed to protect patient anonymity

In [10]:
def segment_transcript_by_movie(patient_id, ecog1_t, ecog2_t):
    # Load in interview transcript
    with open('/scratch/gpfs/mayaar/GravityECoG/sourcedata/interview-transcripts/cleaned/cleaned_transcript_ny' + patient_id + '.csv', newline='') as f:
        reader = csv.reader(f)
        data = list(reader)
        
    # Movie 1
    words_1 = []
    # Movie 2
    words_2 = []

    # Check if word recorded during first or second ECoG recording
    # Do not include words said in first or last 2000ms of either recording (out of sliding time window range)
    for i in range(len(data)):
        if ((float(data[i][2]) > (ecog1_t[0] + 2)) and (float(data[i][2]) < (ecog1_t[-1] - 2))):
            words_1.append(data[i][1])
        if ((float(data[i][2]) > (ecog2_t[0] + 2)) and (float(data[i][2]) < (ecog2_t[-1] - 2))):
            words_2.append(data[i][1])
    
    return words_1, words_2

In [11]:
# Adapted from https://www.geeksforgeeks.org/pre-trained-word-embedding-using-glove-in-nlp-models/

# vocab: 'the': 1, mapping of words with integers in seq. 1,2,3..
# embedding: 1->dense vector
def embedding_for_vocab(filepath, word_index, embedding_dim):
    # Adding again 1 because of reserved 0 index
    vocab_size = len(word_index) + 1
    embedding_matrix_vocab = np.zeros((vocab_size, embedding_dim))
  
    with open(filepath, encoding="utf8") as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word]
                embedding_matrix_vocab[idx] = np.array(vector, dtype=np.float32)[:embedding_dim]
  
    return embedding_matrix_vocab

In [12]:
def get_embeddings(words):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(words)
    print("Number of unique words in dictionary=", len(tokenizer.word_index))
    
    # matrix for vocab: word_index
    embedding_dim = 50
    embedding_matrix_vocab = embedding_for_vocab('glove/glove.6B.50d.txt', tokenizer.word_index, embedding_dim)
    print(embedding_matrix_vocab.shape)
    
    word_embeddings = np.zeros((len(words), 50))

    for i in range(len(words)):
        word_embeddings[i, :] = embedding_matrix_vocab[list(tokenizer.word_index.keys()).index(words[i]) + 1]
        
    return word_embeddings

In [13]:
for patient_id in id_eleccount:
    # Load in ECoG time axis
    ecog1_t = scipy.io.loadmat('/scratch/gpfs/mayaar/GravityECoG/derivatives/preprocessing/sub-ny' + patient_id + '/eeg1_manualica_notch_time.mat')['trial'][0]
    ecog1_t = np.array(ecog1_t, dtype=float)
    ecog2_t = scipy.io.loadmat('/scratch/gpfs/mayaar/GravityECoG/derivatives/preprocessing/sub-ny' + patient_id + '/eeg2_manualica_notch_time.mat')['trial'][0]
    ecog2_t = np.array(ecog2_t, dtype=float)
    
    # Separate words into corresponding ECoG recordings (1 and 2)
    words_1, words_2 = segment_transcript_by_movie(patient_id, ecog1_t, ecog2_t)

    # Generate and save 50 dimensional GloVe embeddings for each word
    embeddings_1 = get_embeddings(words_1)
    embeddings_2 = get_embeddings(words_2)
    np.save("glove_emb_1_" + patient_id + ".npy", embeddings_1)
    np.save("glove_emb_2_" + patient_id + ".npy", embeddings_2)

Number of unique words in dictionary= 410
(411, 50)
Number of unique words in dictionary= 211
(212, 50)
Number of unique words in dictionary= 484
(485, 50)
Number of unique words in dictionary= 297
(298, 50)
Number of unique words in dictionary= 474
(475, 50)
Number of unique words in dictionary= 295
(296, 50)
Number of unique words in dictionary= 528
(529, 50)
Number of unique words in dictionary= 377
(378, 50)
Number of unique words in dictionary= 486
(487, 50)
Number of unique words in dictionary= 283
(284, 50)
Number of unique words in dictionary= 414
(415, 50)
Number of unique words in dictionary= 265
(266, 50)
Number of unique words in dictionary= 466
(467, 50)
Number of unique words in dictionary= 261
(262, 50)
Number of unique words in dictionary= 501
(502, 50)
Number of unique words in dictionary= 316
(317, 50)
Number of unique words in dictionary= 432
(433, 50)
Number of unique words in dictionary= 238
(239, 50)
